### Week 10 Assignment 

Anny Rodrigues Viloria - Fall 2020

* For this week's assignment I am focusing on exploring the data for the country I am focusing on: Colombia. The data I am using here comes from the Demographic and Health Survey (DHS) made available by the 
[USAID Spatial Data Repository](http://spatialdata.dhsprogram.com/home/). The survey is conducted every year and I downloaded all the data available from 1985 to 2015.

In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
# importing all my data
col_15=gpd.read_file('shps_colombia/sdr_subnational_data_dhs_2015.shp')
col_10=gpd.read_file('shps_colombia/sdr_subnational_data_dhs_2010.shp')
col_05=gpd.read_file('shps_colombia/sdr_subnational_data_dhs_2005.shp')
col_00=gpd.read_file('shps_colombia/sdr_subnational_data_dhs_2000.shp')
col_95=gpd.read_file('shps_colombia/sdr_subnational_data_dhs_1995.shp')
col_90=gpd.read_file('shps_colombia/sdr_subnational_data_dhs_1990.shp')
col_86=gpd.read_file('shps_colombia/sdr_subnational_data_dhs_1986.shp')

In [ ]:
# concatenating the file so it's a single file instead of one per year
frames = [col_86,col_90,col_95,col_00,col_05,col_10,col_15]
col = pd.concat(frames)

In [ ]:
type(col)

In [ ]:
# i expect to see multiple years in the date column to confirm concatenation worked
col.sample(5)

In [ ]:
# to get a sense of the column names, data types, and missing values:
col.info(verbose=True, null_counts=True)

from the above table I can see there are too many extra colums to i'll proceed to remove them from my dataframe

In [ ]:
df=col[['SVYYEAR','DHSREGSP','CMECMRCIMR','CMECMRCU5M','EDEDUCWSEH','EDLITRWLIT','MMMMROWMMR','MMMMROWPMR','MMMMRTWFDM','MMMMRTWFDP','MMMMRTWMDT','MMMMRTWPDT','RHDELPCDHF','geometry']]

In [ ]:
df.sample(10)

well, this is a bummer. most of my columns are empty. however, there is data for infant mortality rate, under-5 mortality, women's education, and women's place of delivery. two of these columns are the primary outcomes of interest for my project which is great, but i will need to find maternal mortality data elsewhere. for now, i'll cut down the columsns again and work to visualize that data.

In [ ]:
df=col[['SVYYEAR','DHSREGSP','CMECMRCIMR','CMECMRCU5M','EDEDUCWSEH','RHDELPCDHF','geometry']]
df.columns=['year','region','infantmort','u5mort','womened','birthplace','geometry']
df.sample(5)

In [ ]:
#sorting dataframe by year
df.sort_values(by='year',ascending=True)

two things to note here: the index needs to be reset, and the older DHS surveys do not contain data on women's m education or place of delivery. this feels counterintuitive as i would have expected to have more social outcomes reported in the past than today, when i assume these indicators to be overall high.

In [ ]:
df.reset_index()

### data visualizations

In [ ]:
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [ ]:
# becuase i want to create an interactive map, i will start by listing the fields i want to use for the @interact function
year_list = df.year.unique().tolist()
year_list

In [ ]:
region_list = df.region.unique().tolist()
region_list

In [ ]:
indicator_list = ['infantmort','u5mort','womened','birthplace']

In [ ]:
# reproject
df = df.to_crs('EPSG:3857')
# get the centroid x and y's into their own columns
df['x'] = df.geometry.centroid.x
df['y'] = df.geometry.centroid.y

In [ ]:
# import pyplot and contextily
import matplotlib.pyplot as plt
import contextily as ctx

In [ ]:
@interact

def viz_by(indicator=indicator_list, year=year_list):
    #Set up figure and axis
    f, ax = plt.subplots(1, figsize=(20,10))
    #map
    df.plot(figsize=(50,50),
                 column=indicator,
                 legend=True, 
                 scheme='NaturalBreaks',
                 ax=ax)

    # axis
    ax.axis('off')

    # basemap
    ctx.add_basemap(ax,source=ctx.providers.CartoDB.Voyager)

I wasn't sure what to do to incorporate the year dropdown, i tried a million combinations and nothing seemed to work.

In [ ]:
# Set up figure and axis
f, ax = plt.subplots(1, figsize=(10,10))

# kdeplot, i.e. heat map
sns.kdeplot(x = df.x, 
            y = df.y,
            n_levels=100, #number of circles
            shade=True,
            thresh=0.05, #this is what % of circles if shown, in this case 1-.05=95% of data dropped
            alpha=0.2, #this is opacity of the color
            cmap='Reds')

# axes
ax.axis('off')

# basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.DarkMatter)

i think the above map doesn't atually tell much because i would first need to cut the dataframe to have one indicator only. i'll be working on this and include it on the final project.